<a href="https://colab.research.google.com/github/PrinceP/MonoRepo/blob/dl_learning/src/colabs/assignments/Submission_Painting_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment link: https://www.notion.so/playment/Playment-ML-Engineer-Assignment-Segmentation-04bbcb8c4b474603829e531aa5fbe852

Create a copy of this Colab file, fill it in, train your model and share along with results

# Overview

This notebook implements the Detectron2 (https://github.com/facebookresearch/detectron2) library on the Bob Ross Paintings dataset to create semantic segmentation masks.

Detectron2 tutorial Colab: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=h9tECBQCvMv3 

# Dataset overview

Dataset link: https://www.kaggle.com/residentmario/segmented-bob-ross-images

This dataset contains ~250 segmented Bob Ross paintings with 9 classes. Bob Ross was a painter and painting instructor who was on PBS public television for over a decade with his show "The Joy of Painting". Bob Ross is known for his easy-to-learn "wet-on-wet" painting style, the use of vibrant color in his landscape paintings, and for his generally calm, joyous personality.

## Sample images

![picture](https://drive.google.com/uc?export=view&id=1htccDGLye-d-XtokQt-9UXUDqw64IBt_)
![picture](https://drive.google.com/uc?export=view&id=1-52cAFtqUeqebVegcrza6yHvTjkOchyT)

## Sample masks

![picture](https://drive.google.com/uc?export=view&id=1uamJLZRYCY7Rj-dejTl_vlSfo88yBQci)
![picture](https://drive.google.com/uc?export=view&id=1p_12HaEfSCf4PAzNjXLmyUaj-cqlYOid)

In [ ]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 5.1 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=8f4a9f31936ddac18665524cb233e76cfb125e584c6a1f3ae61c0321141d7422
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
     |████████████████████████████████| 6.4 MB 602 kB/s 
     |████████████████████████████████| 74 kB 2.4 MB/s 
     |████████████████████████████████| 145 kB 11.0 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 130 kB 64.5 MB/s 
     |████████████████████████████████| 747 kB 57.7 MB/s 
     |████████████████████████████████| 743 kB 58.2 MB/s 
     |██

In [ ]:
# check pytorch installation: 
# Make sure your runtime type is GPU
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.9.0+cu102 False


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import pprint
import copy

# import required detectron2 utilities


# Train on a custom dataset

In this section, we will train an existing detectron2 model on a custom dataset which is not in the detectron2 format.

We'd love to see how you can improve the performance of the model. 

# Downloading the dataset
You will need to upload your kaggle API keys (kaggle.json file) to the Colab folder. 

Please make sure to select the GPU runtime under Runtime --> Change runtime type.

In [ ]:
! mkdir -p ~/.kaggle 
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download residentmario/segmented-bob-ross-images
! unzip -n segmented-bob-ross-images

Register the Bob Ross Paintings dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we need to write a function to parse it and prepare it into detectron2's standard format. User should write such a function when using a dataset in custom format. See the tutorial for more details.


In [ ]:
IMAGE_DIR = 
MASK_DIR = 
REM_MASK_DIR = 

The labels.csv file has non contiguous values for the labels. 
![picture](https://drive.google.com/uc?export=view&id=1NS1DhrQykH0ta2p6DhEe4tMKWNfV1tZr)

Let's convert these into contiguous values for ease of visualization and evaluation

In [ ]:
def remap_mask_labels_to_contiguous_ints():
  
  # Your code goes here
  # Create new mask files after remap and place them in REM_MASK_DIR

In [ ]:
def train_val_split(IMAGE_DIR, ):

    # Your code goes here

    return (train_files, val_files)

In [ ]:
def get_painting_dicts(filenames):
    
    # Your code goes here

    return dataset_dicts

In [ ]:
def reset():
    DatasetCatalog.clear()
    MetadataCatalog.clear()

---
Keep all functions above this line
---

In [ ]:
reset()

remap_mask_labels_to_contiguous_ints()

In [ ]:
train_files, val_files = train_val_split(IMAGE_DIR)

for d, files in [("train", train_files), ("val", val_files)]:
    # Register Catalogs


To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:

In [ ]:
# Visualize 3 random samples from the dataset

## Train!


In [ ]:
cfg = get_cfg()

# Your code goes here
# Configure the trainer and start training

In [ ]:
# # Look at training curves in tensorboard:
# %load_ext tensorboard
# %tensorboard --logdir output

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the painting validation dataset. First, let's create a predictor using the model we just trained:

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We should change it a little bit for inference:

# cfg.

In [ ]:
# Visualize inference of 3 random samples 

In [ ]:
# Evaluate metrics and print

# pp = pprint.PrettyPrinter()

#evaluator = 
#val_loader = 
#pp.pprint(inference_on_dataset(predictor.model, val_loader, evaluator))